In [2]:
import os 
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from chat_memory import get_chat_history, last_n_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
import textwrap

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
vectorstore = Chroma(persist_directory="path/to/your/chroma_db", embedding_function=embeddings)

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Router

In [4]:
router_prompt = ChatPromptTemplate.from_messages([
    ("system", """Your job is to classify user queries into two buckets: 
     - memory
     - memory and vectorstore
     
     *Guidelines*
     Questions regarding exercises, exercise form, or technique: vectorstore & memory
     General question or statements: memory
   
     *Respond to queries with only the correct class*
     Examples: 
     User: how should I grip the bar for bench press?
     Agent: vectorstore & memory

     User: what muscles should I feel during the Romanian dead lift?
     Agent: vectorstore & memory

     User: How's this? 
     Agent: vectorstore & memory

     User: Is this good squat technique? 
     Agent: vectorstore & memory

     User: Was my bar path better?
     Agent: vectorstore & memory

     User: thakns for your help!
     Agent: memory

"""),

     ("human", "{query}")
     
])

llm = ChatOpenAI(model='gpt-4o')

output_parser = StrOutputParser()

router_chain = router_prompt | llm | output_parser

router_response = router_chain.invoke({"query": "Looking forward to working with you! My name is Chandler."})

print(router_response)

memory


In [7]:
def router(user_query):
    route = router_chain.invoke({"query": user_query})
    route = route.lower().strip()
    return route


router("thanks for everything!")

'memory'

# Orchestrator

In [8]:
 
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a world-class fitness coach. You have extensive experience in helping weight lifters achieve perfect form and maximum hypertrophy. 
Your job is to analyze images of users lifting weights, offer them advice from your context, and to answer any questions they might have. 
Inspect each image CLOSELY and arefuly for problems or issues related to best practices in exercise form. Help the user diagnose their incorrect form. 
Be specific about what you observe.

# ANSWER CONTEXT
Use ONLY the following context when answering a user: 
     
---   
{context}
 ---
"""),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{query}")
])

llm = ChatOpenAI(model='gpt-5',
                 temperature=0.5)

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"

)

response = chain_with_history.invoke(
    {"query": "what's good grip for bench press?", "context": ""},
    config={"configurable": {"session_id": "test_123"}}
)

print(textwrap.fill(response))

print(response)

Great question. A “good” bench press grip is the one that keeps your
wrists stacked over your elbows at the bottom, lets you touch the bar
to your lower chest comfortably, and feels strong on your shoulders.
Here’s how to dial it in:  How to find your grip - Start width: Put
your pinky or ring finger on the bar’s knurling rings. That’s a solid
starting point for most. Adjust 1–2 finger-widths in or out based on
the checks below. - Bottom position check: With the bar on your lower
chest, your forearms should be vertical from the front view—wrist
directly over elbow.    - If your wrists sit inside (toward your
torso) of your elbows, your grip is too narrow.   - If your wrists sit
outside your elbows, your grip is too wide. - Elbow angle: About 45–75
degrees from your torso at the bottom (not flared straight out, not
tucked to your ribs). - Touch point: Lower chest/sternum, not upper
chest.  Hand and wrist cues - Bar in the heel of your palm with
knuckles up; keep wrists neutral (don’t le

In [9]:
def handle_query(user_query, session_id): # create function for handling queries 
    route = router(user_query) # pass route variable into function ith router function
    print(route)

    if "vectorstore" in route: # if route contains vectorstore 
        results = vectorstore.similarity_search(user_query, k=4) # pass the user query to similariy search (vectorstore)
        context = "\n".join([r.page_content for r in results]) # pass each of the results (k=n), joined, into the context to be read by image analyzer
    else: 
        context = "" # context is blank but we stil pass the query


    # now call the LLM with memory
    response = chain_with_history.invoke( 
        {"query": user_query, "context": context}, # pass the user query and the context to the chat_memory (vectorstore & memory)
        config={"configurable": {"session_id": session_id}} # initalize or call the chat's session_id
    )
    return response # return a resopnse that uses vectorstore & memory

In [10]:
handle_query("what's a good grip for bench press?", "test_456")

vectorstore & memory


'Great question. Without seeing your setup, here’s a reliable way to dial in a strong, safe bench grip.\n\nBaseline grip\n- Width: Start slightly wider than shoulder width. A practical start is pinkies on the bar’s outer rings (moderate). Adjust 1–2 finger-widths wider/narrower based on the checkpoints below.\n- Thumb: Use a full, wrapped grip (thumb around the bar) for safety.\n- Hand placement: Sit the bar low in your palm (toward the base of the thumb), not in the fingers. Knuckles toward the ceiling.\n- Wrist: Stack wrist directly over the elbow at the bottom; minimal backward bend.\n\nForm checkpoints (film from front and side if possible)\n- Bottom position: Forearms should be vertical when the bar touches the lower chest/upper sternum. If forearms angle in, you’re too wide; if they angle out, you’re too narrow.\n- Elbow angle: About 45–70 degrees from your torso (not flared straight out).\n- Bar path: Down to lower chest/sternum, up toward shoulder line (slight J-curve).\n- Symm

In [11]:
handle_query("I've seen some guys grip the bar without warpping their whole hand around it. What is that? Should I try it?", "test_456")

vectorstore & memory


'You’re seeing a thumbless/“false” grip. The thumb doesn’t wrap around the bar; the bar sits across the palm with the thumb on the same side as the fingers. Some lifters also use a “bulldog” variant where the bar is set very low in the palm with the wrist stacked; this can be done with the thumb wrapped or thumbless.\n\nShould you try it?\n- My default: No for bench. A full, wrapped grip is safer and just as strong for most lifters.\n- Risks: With thumbless bench the bar can roll out if it drifts into your fingers, you lose some tactile control, and many federations don’t allow it in competition. The risk spikes when you’re fatigued, sweaty, or using a thick/slippery bar.\n- Possible reason people use it: It can reduce wrist extension and feel better on wrists if they’ve been loading the bar too high in the hand. But you can usually fix that without going thumbless.\n\nSafer alternatives that give the same benefits\n- Use a wrapped “bulldog” grip:\n  - Place the bar low in your palm ne

In [12]:
handle_query("sounds good!", "test_456")

memory


'Perfect. Send two short clips at your working weight and I’ll dial everything in.\n\nHow to film (10–15s each)\n- Front angle: 30–45° off center, camera at mid‑chest height, 6–10 ft away. Include the whole bar, your torso, and elbows.\n- Side angle: Perpendicular to the bench, camera at mid‑chest height. Capture feet-to-bar so I can see leg drive, arch, butt/shoulder contact, bar path.\n\nWhat to show\n- 3–5 reps at normal tempo, include the unrack and re‑rack.\n- A 2–3s close‑up of your hand placement on the knurling before the set.\n- Keep plates and collars visible so I can confirm load.\n\nWhat to tell me\n- Exact grip (which finger on the power rings), bar weight, and whether you pause or touch‑and‑go.\n- Any discomfort (wrists, elbows, shoulders) and your main goal (strength, hypertrophy, shoulder‑friendly, etc.).\n\nSafety\n- Use a spotter or set safeties just below chest height when the bar is on you.\n- Use collars.\n\nIf you can’t film, I can still help—tell me your shoulder

In [13]:
handle_query("what about my foot placement?", "test_456")

vectorstore & memory


'Great question—foot placement makes or breaks your leg drive and upper‑back stability on bench.\n\nWhat to aim for\n- Stance width: About shoulder-width to slightly wider.\n- Toes: Turned out 15–30° so you can “screw” feet into the floor.\n- Position: Feet slightly behind your knees. Shins vertical to slightly angled back at the bottom. This lets you drive back toward your head without your butt popping up.\n- Contact: Full “tripod” on each foot—big toe, little toe, and heel glued to the floor the entire set.\n- Direction of force: Push the floor away/back toward your head to wedge your shoulders harder into the bench, not straight up.\n\nSimple setup (30 seconds)\n1) Set your shoulder blades down and back.\n2) Plant feet where you can keep full tripod contact; angle them slightly behind your knees.\n3) Pre‑load leg drive by gently pushing back toward your head until you feel quad tension and your upper back locks in.\n4) Unrack without losing that pressure. Keep steady leg drive thro